# Road Traffic Accidents in Switzerland

Our project goal is to scrap all traffic accidents from the accidents map from http://map.donneesaccidents.ch/

## Data scraping strategy

Accessing http://map.donneesaccidents.ch/, wich redirects to : <br>
https://map.geo.admin.ch/?topic=vu&lang=fr&bgLayer=ch.swisstopo.pixelkarte-grau&layers=ch.astra.unfaelle-personenschaeden_alle&layers_timestamp=&catalogNodes=1318


Postman parses the following parameters : 
<code>
topic:vu
lang:en
bgLayer:ch.swisstopo.pixelkarte-grau
layers:ch.astra.unfaelle-personenschaeden_alle
layers_timestamp:
catalogNodes:1318
</code>

The most important one is layers:ch.astra.unfaelle-personenschaeden_alle.<br>
It is the layer that contains all the geo-information dots on "Accidents with personal injury" which is the selected data layer.
<img src="layer_selector.png">

Selection all kinds of accidents returns the following :<br>
<img src="layer_selector_all.png">
with layer parameters :<br>
layers:<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_alle,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_getoetete,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fussgaenger,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fahrraeder,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_motorraeder<br>
layers_timestamp:,,,,<br>

Now we want every data for each layer. By selecting a dot on the map, it queries the related data to the server.
What we want to do is selecting all the entries in the map to retrieves all data. This is done by ctrl clicking the whole area.

This makes a query for each "layers" parameter :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:<i>LAYER_PARAM</i>
mapExtent:269999.04209536605,9501.673331833561,1037999.042095366,389001.67333183356
returnGeometry:true
tolerance:5
</code><br>
But doesn't select all dots on map, so let's try the "load more results" button on a 'accidetns with fatalities' layer, we get :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:ch.astra.unfaelle-personenschaeden_getoetete
mapExtent:136199.04209536605,-28148.32666816644,1134599.042095366,465201.67333183356
<b>offset:200</b>
returnGeometry:true
tolerance:5
</code>
Pressing load more until no more possible give offset=1200 (for a total of 1337 objects) i.e it loads data entries 200 by 200

## JSON Data scraping

In [1]:
import requests
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from Scripts.helpers import *
from Scripts.plots import *


import pprint
#from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
#import raw data
data = import_data(all_data = False)

Processing layer : ch.astra.unfaelle-personenschaeden_alle
Layer processed : 201 records

Whole dataset processed : 201 records



In [4]:
#translate data from german
json_data_preprocessed = preprocess_data(data)

In [5]:
print("Data entry example after clean and reformat:\n")
json_data_preprocessed[0]

Data entry example after clean and reformat:



{'accidentday_fr': 'mercredi / 12h-13h / octobre 2014',
 'accidenttype_fr': 'dérapage ou perte de maîtrise',
 'accidenttypecode': 0,
 'accidentyear': 2014,
 'bbox': [673752.0, 250106.0, 673752.0, 250106.0],
 'canton': 'ZH',
 'coordinates': [[673752.0, 250106.0]],
 'featureId': '05E9B741E6040188E0530A839427C016',
 'fsocommunecode': '0243',
 'geometryType': 'Feature',
 'id': '05E9B741E6040188E0530A839427C016',
 'label': 'Schleuder- oder Selbstunfall',
 'layerBodId': 'ch.astra.unfaelle-personenschaeden_alle',
 'layerName': 'Accidents avec dommages corporels',
 'roadtype_fr': 'route secondaire',
 'roadtypecode': 433,
 'severitycategory_fr': 'accident avec blessés légers',
 'severitycategorycode': 'ULV',
 'type': 'Feature'}

In [6]:
df = pd.DataFrame.from_dict(json_data_preprocessed)
df.set_index('id', inplace=True)
df.head()

,accidentday_fr,accidenttype_fr,accidenttypecode,accidentyear,bbox,canton,coordinates,featureId,fsocommunecode,geometryType,label,layerBodId,layerName,roadtype_fr,roadtypecode,severitycategory_fr,severitycategorycode,type
id,,,,,,,,,,,,,,,,,,
05E9B741E6040188E0530A839427C016,mercredi / 12h-13h / octobre 2014,dérapage ou perte de maîtrise,0,2014,"[673752.0, 250106.0, 673752.0, 250106.0]",ZH,"[[673752.0, 250106.0]]",05E9B741E6040188E0530A839427C016,0243,Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,route secondaire,433,accident avec blessés légers,ULV,Feature
05E9B939FE3E0116E0530A839427B6FD,mercredi / 06h-07h / octobre 2014,accident en quittant une route,3,2014,"[831057.0, 169150.0, 831057.0, 169150.0]",GR,"[[831057.0, 169150.0]]",05E9B939FE3E0116E0530A839427B6FD,3847,Feature,Abbiegeunfall,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,route principale,432,accident avec blessés légers,ULV,Feature
05E9B93A00AD0116E0530A839427B6FD,mercredi / 15h-16h / octobre 2014,dérapage ou perte de maîtrise,0,2014,"[683001.0, 248929.0, 683001.0, 248929.0]",ZH,"[[683001.0, 248929.0]]",05E9B93A00AD0116E0530A839427B6FD,0261,Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,semi-autoroute,431,accident avec blessés légers,ULV,Feature
0612A7B27C56017CE0530A839427BC68,mercredi / 06h-07h / octobre 2014,accident par tamponnement,2,2014,"[674274.0, 251041.0, 674274.0, 251041.0]",ZH,"[[674274.0, 251041.0]]",0612A7B27C56017CE0530A839427BC68,0251,Feature,Auffahrunfall,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,autoroute,430,accident avec blessés légers,ULV,Feature
0612A7B27F9A017CE0530A839427BC68,mercredi / 17h-18h / octobre 2014,accident par tamponnement,2,2014,"[686475.0, 256393.0, 686475.0, 256393.0]",ZH,"[[686475.0, 256393.0]]",0612A7B27F9A017CE0530A839427BC68,0062,Feature,Auffahrunfall,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,route principale,432,accident avec blessés légers,ULV,Feature


In [7]:
for feature in df : 
    print("plotting feature {}".format(feature))
    plot_feature(df, feature)
print("Done plotting")

plotting feature accidentday_fr
plotting feature accidenttype_fr
plotting feature accidenttypecode
plotting feature accidentyear
plotting feature bbox
plotting feature canton
plotting feature coordinates
plotting feature featureId
plotting feature fsocommunecode
plotting feature geometryType
plotting feature label
plotting feature layerBodId
plotting feature layerName
plotting feature roadtype_fr
plotting feature roadtypecode
plotting feature severitycategory_fr
plotting feature severitycategorycode
plotting feature type
Done plotting


# Data analysis

1) Accidents par rapport au temps<br>
2) Corrélation nombre/type d'accident avec les endroits (Valais ivresse)<br>
3) Tracker des anomalies (fin/début d'une série d'accident) et essayer d'en trouver la cause<br>